In [639]:
import pandas as pd
import re
import usaddress
import plotly.graph_objects as go

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [640]:
df = pd.read_csv('csv_data.csv', encoding="utf-8")

In [641]:
df.size

38619

In [642]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1839 entries, 0 to 1838
Data columns (total 21 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Date Added                                         1839 non-null   object 
 1   category                                           1839 non-null   object 
 2   sellerlink                                         1839 non-null   object 
 3   sellerlink-url                                     1839 non-null   object 
 4   sellerstorefront-url                               1839 non-null   object 
 5   sellerproductcount                                 1513 non-null   object 
 6   sellerratings                                      1411 non-null   object 
 7   sellerdetails                                      1812 non-null   object 
 8   seller business name                               1783 non-null   object 
 9   business

In [643]:
# Dropping unnamed extra columns
df.drop(['Unnamed: 18', 'Unnamed: 19','Unnamed: 20'], axis=1, inplace=True)

In [644]:
# Describe the dataframe
df.describe()

,Count of seller brands,Max % of negative seller ratings - last 30 days,Max % of negative seller ratings - last 90 days,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings
count,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000
mean,6.951060,5.808592,7.632409,12.312670,1399.745514,765.173464
std,6.232073,18.117027,19.902394,23.435871,6204.123987,3817.355945
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.000000,0.000000,0.000000,0.000000,3.000000,1.000000
75%,15.000000,0.000000,4.000000,14.000000,176.500000,87.500000
max,16.000000,100.000000,100.000000,100.000000,86856.000000,70732.000000


### <span style="color:red">Extract Product Count</span>

In [646]:
def parse_product_count(_str):
    pattern = '([\d]+(,)*[\d]*)+ results'
    
    # Check if NaN
    if _str != _str:
        return
    
    result = re.search(pattern,_str)
    if result:
        # Extract numerical string
        product_count = result.group(0).split(' ')[0]
        product_count = int(product_count.replace(',',''))
        return product_count

In [647]:
# Create a new column product_count
df['product_count'] = df.apply(lambda x: parse_product_count(x.sellerproductcount),axis=1)

In [648]:
# Print a sample of extracted data
df[['product_count','sellerproductcount']].head()

,product_count,sellerproductcount
0,100000.0,"1-16 of over 100,000 results"
1,NaN,NaN
2,2000.0,"1-16 of over 2,000 results"
3,123.0,1-16 of 123 results
4,1000.0,"1-16 of over 1,000 results"


In [649]:
# Check if any sellerproductcount has not been parsed
df[['product_count','sellerproductcount']].loc[(df['product_count'].isna() & df['sellerproductcount'].notna())]

,product_count,sellerproductcount


In [650]:
# Check if any product_count has been incorrectly extracted
df[['product_count','sellerproductcount']].loc[(df['product_count'].notna() & df['sellerproductcount'].isna())]

,product_count,sellerproductcount


------

### <span style="color:red">Extract No. of Seller ratings and Positive ratings %</span>

In [651]:
def parse_seller_ratings(_str):
    rating_count_pattern = '[\d]+ ratings'
    positive_rating_pattern = '[\d]+%'
    
    # Check if NaN
    if _str != _str:
        return
    
    rating_count = re.search(rating_count_pattern,_str)
    positive_rating = re.search(positive_rating_pattern,_str)
    
    if rating_count:
        rating_count = rating_count.group(0)
        rating_count = int(rating_count.split(' ')[0].replace(',',''))
    if positive_rating:
        positive_rating = int(positive_rating.group(0).replace('%',''))
        
    return (rating_count,positive_rating)

In [652]:
parse_seller_ratings('90% positive in the last 12 months (566 ratings)')

(566, 90)

In [653]:
# Create two new columns rating_count and positive_rating_perc
# df[['rating_count','positive_rating_perc']] = df.apply(lambda x: parse_seller_ratings(x.sellerratings),axis=1)
df[['rating_count','positive_rating_perc']] = df.apply(lambda x: parse_seller_ratings(x.sellerratings), axis=1, result_type='expand')



In [685]:
df.head(5)

,Date Added,category,sellerlink,sellerlink-url,sellerstorefront-url,sellerproductcount,sellerratings,sellerdetails,seller business name,businessaddress,Count of seller brands,Max % of negative seller ratings - last 30 days,Max % of negative seller ratings - last 90 days,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,Sample brand name,Sample Brand URL,product_count,rating_count,positive_rating_perc,email,phone_number,placename,statename,zipcode,country
0,15-Nov-20,Garden,Seller 1,Seller 1-link,Seller 1-storefrontlink,"1-16 of over 100,000 results",88% positive in the last 12 months (118 ratings),"Lohas Living Inc James Mazzello US 845 3RD Ave Fl 6 New York NY 10022-6630 +1208-964-6807 Email Address:jadgemaello@gmail.com James Mazzello US 845 3RD Ave Fl 6 New York NY 10022-6630 +1208-964-6807 EIN: 30-0961431 \nThe European Commission provides for an Online Dispute Resolution Platform, which you can access here: https://ec.europa.eu/consumers/odr/\n\nPlease see the following link for the nationally appointed Alternative Dispute Resolution bodies contact details: https://webgate.ec.europa.eu/od...",Business Name:Lohas Living Inc,411 THEODORE FREMD AVESTE 206 SOUTHRyeNY10580-1410US,16,17,11,12,5873,4957,Visit the YAMAZAKI Store,https://www.amazon.de/-/en/stores/YAMAZAKI/page/B9975298-FF1F-4EE2-BD7B-53AE47A44D8A?ref_=ast_bln,100000.0,118.0,88.0,jadgemaello@gmail.com,1208-964-6807,New York New York,NY,10022-6630,US
1,15-Nov-20,Garden,Seller 2,Seller 2-link,Seller 2-storefrontlink,NaN,90% positive in the last 12 months (566 ratings),"Herzlich Willkommen im Amazon Shop von 1a-Handelsagentur. Für Fragen stehen wir Ihnen gerne unter der 039932-829721 zur Verfügung. Sie können auch direkt eine E-Mail an info@1a-handelsagentur.de schreiben. Anschrift\n1a-handelsagentur.de\nAndre Burmeister\nLindenallee 2\n17213 Malchow Kontakt\nTel.: 039932 - 829721 \nFax.: 039932 – 81417 \nE-Mail: info@1a-handelsagentur.de\nInternet: www.1a-handelsagentur.de\n\nUmsatzsteueridentnummer: DE 223758473 \nWir berechnen Ihnen folgende Versandkosten:(DE) Deutschland 0,00 E...",Business Name:1a-Handelsagentur,1a HandelsagenturLindenallee 2MalchowMecklenburg17213DE,0,2,2,9,0,0,Visit the tesa Store,https://www.amazon.de/-/en/stores/tesa/page/12E699E9-FA20-4BC9-A12B-8A9CDE0C446C?ref_=ast_bln,NaN,566.0,90.0,info@1a-handelsagentur.de,932-8297,None,None,None,DE
5,15-Nov-20,Garden,Seller 6,Seller 6-link,Seller 6-storefrontlink,1-16 of 28 results,NaN,7830jeff is committed to providing each customer with the highest standard of customer service.,NaN,NaN,13,0,0,0,3284,2944,Strata,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0?ie=UTF8&search-type=ss&index=garden-de&field-keywords=Strata,28.0,NaN,NaN,None,None,None,None,None,None
14,15-Nov-20,Garden,Seller 15,Seller 15-link,Seller 15-storefrontlink,1-16 of 919 results,97% positive in the last 12 months (116 ratings),"Impressum\n \n Gesetzliche Anbieterkennung:\n \n Oliver Mills\n 9:PM\n An der Bundesstrasse 26\n 33829 Borgholzhausen\n Deutschland\n Telefon: 015140008562\n E-Mail: webmaster@9pm-store.de\n USt-IdNr.: DE215752000\n \n Wir sind seit 01.08.2015 Mitglied der Initiative ""FairCommerce"".\n Nähere Informationen hierzu finden Sie unter .fair-commerce.de. Alternative Streitbeilegung: \nDie Europäische Kommission bietet eine Onlineplattform für Streitbeilegung an, die Sie hier finden: https://ec.europa.eu/consumers/odr/\n\nUnter diesem L...",Business Name:Oliver Mills,Kaiserstr.12Borgholzhausen33829DE,15,0,0,3,1446,921,Unbekannt,https://www.amazon.de/-/en/Unbekannt/b/ref=bl_dp_s_web_19533614031?ie=UTF8&node=19533614031&field-lbr_brands_browse-bin=Unbekannt,919.0,116.0,97.0,webmaster@9pm-store.de,None,None,None,None,DE
19,15-Nov-20,Garden,Seller 20,Seller 20-link,Seller 20-storefrontlink,NaN,76% positive in the last 12 months (3721 ratings),"Impressum: ABC-Schnäppchenmarkt GmbH \n Jösser Weg 10\n 32469 Petershagen-Windheim\n Deutschland \n \n Tel.: 0570591155\n Fax: 057

### <span style="color:red"> Parse Seller Details </span>

In [655]:
pd.set_option('display.max_colwidth', None)

In [656]:
df['sellerdetails'].head(1)

0    Lohas Living Inc James Mazzello US 845 3RD Ave Fl 6 New York NY 10022-6630 +1208-964-6807  Email Address:jadgemaello@gmail.com James Mazzello  US 845 3RD Ave Fl 6 New York NY 10022-6630 +1208-964-6807 EIN: 30-0961431 \nThe European Commission provides for an Online Dispute Resolution Platform, which you can access here: https://ec.europa.eu/consumers/odr/\n\nPlease see the following link for the nationally appointed Alternative Dispute Resolution bodies contact details: https://webgate.ec.europa.eu/od...
Name: sellerdetails, dtype: object

In [657]:
def parse_phone_number(_str):
#     phone_number_pattern = '(\d{3,4}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})'
    phone_number_pattern = '(\d{3,4}[-\.\s]+\d{3,4}[-\.\s]+\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]\d{4})'
    if _str != _str:
        return
    phone_no = re.search(phone_number_pattern,_str)
    if phone_no:
        phone_no = phone_no.group(0)
        return phone_no
    
def parse_email(_str):
    email_pattern = '[\w.+-]+@[\w-]+\.[\w.-]+'
    if _str != _str:
        return
    email = re.search(email_pattern,_str)
    if email:
        email = email.group(0)
        return email

def parse_seller_details(_str):
    if _str != _str:
        return
    parsed = usaddress.parse(_str)
    placename_list = []
    
    statename = None
    zipcode = None
    phone_number = None
    email = None
    
    for item in parsed:
        if item[1] == 'StateName':
            statename = item[0]
        if item[1] == 'ZipCode':
            zipcode = item[0]
        if item[1] == 'PlaceName':
            placename_list.append(item[0])
        if item[1] == 'Recipient'and parse_phone_number(item[0]):
            phone_number = parse_phone_number(item[0])
        if item[1] == 'Recipient'and parse_email(item[0]):
            email = parse_email(item[0])
    
    if len(placename_list) > 0:
        placename = ' '.join(placename_list)
    else:
        placename = None
        
    return (email,phone_number,placename,statename,zipcode) 
            

In [658]:
def parse_country(_str):
    if _str != _str:
        return
    country = _str[-2:]
    if country:
        return country

In [659]:
# Extract seller data
df[['email','phone_number','placename','statename','zipcode']] = df.apply(lambda x: parse_seller_details(x.sellerdetails), axis=1, result_type='expand')


In [660]:
# Extract country
df['country'] = df.apply(lambda x: parse_country(x.businessaddress), axis = 1)

In [661]:
# df[['sellerdetails','email','phone_number','placename','statename','country','zipcode']]
# df[df.sellerdetails.str.contains('United Kingdom',na=False,case=False)]
# parse_address('411 THEODORE FREMD AVESTE 206 SOUTHRyeNY10580-1410US')
# df['businessaddress'].head(1)

In [662]:
# Country-wise count
df.groupby('country').size()

country
AT       8
AU       1
BE       1
CH       1
CN    1237
CZ       2
DE     364
ES      15
FR      11
GB      47
HK      10
IE       2
IN       4
IT      25
JP       2
NL      12
PL       5
PT       1
SE       1
TH       1
US      14
dtype: int64

In [663]:
# Filter out CN region businesses
df = df[df.country != 'CN']
df.groupby('country').size()

country
AT      8
AU      1
BE      1
CH      1
CZ      2
DE    364
ES     15
FR     11
GB     47
HK     10
IE      2
IN      4
IT     25
JP      2
NL     12
PL      5
PT      1
SE      1
TH      1
US     14
dtype: int64

In [664]:
# parse_seller_details(str(df['sellerdetails'].head(1)))

In [665]:
# parse_seller_details('304 North Avenue Manhattan New York US')

In [666]:
# usaddress.parse(str(df['sellerdetails'].head(1)))

### Final DF

In [667]:
df.head(5)

,Date Added,category,sellerlink,sellerlink-url,sellerstorefront-url,sellerproductcount,sellerratings,sellerdetails,seller business name,businessaddress,Count of seller brands,Max % of negative seller ratings - last 30 days,Max % of negative seller ratings - last 90 days,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,Sample brand name,Sample Brand URL,product_count,rating_count,positive_rating_perc,email,phone_number,placename,statename,zipcode,country
0,15-Nov-20,Garden,Seller 1,Seller 1-link,Seller 1-storefrontlink,"1-16 of over 100,000 results",88% positive in the last 12 months (118 ratings),"Lohas Living Inc James Mazzello US 845 3RD Ave Fl 6 New York NY 10022-6630 +1208-964-6807 Email Address:jadgemaello@gmail.com James Mazzello US 845 3RD Ave Fl 6 New York NY 10022-6630 +1208-964-6807 EIN: 30-0961431 \nThe European Commission provides for an Online Dispute Resolution Platform, which you can access here: https://ec.europa.eu/consumers/odr/\n\nPlease see the following link for the nationally appointed Alternative Dispute Resolution bodies contact details: https://webgate.ec.europa.eu/od...",Business Name:Lohas Living Inc,411 THEODORE FREMD AVESTE 206 SOUTHRyeNY10580-1410US,16,17,11,12,5873,4957,Visit the YAMAZAKI Store,https://www.amazon.de/-/en/stores/YAMAZAKI/page/B9975298-FF1F-4EE2-BD7B-53AE47A44D8A?ref_=ast_bln,100000.0,118.0,88.0,jadgemaello@gmail.com,1208-964-6807,New York New York,NY,10022-6630,US
1,15-Nov-20,Garden,Seller 2,Seller 2-link,Seller 2-storefrontlink,NaN,90% positive in the last 12 months (566 ratings),"Herzlich Willkommen im Amazon Shop von 1a-Handelsagentur. Für Fragen stehen wir Ihnen gerne unter der 039932-829721 zur Verfügung. Sie können auch direkt eine E-Mail an info@1a-handelsagentur.de schreiben. Anschrift\n1a-handelsagentur.de\nAndre Burmeister\nLindenallee 2\n17213 Malchow Kontakt\nTel.: 039932 - 829721 \nFax.: 039932 – 81417 \nE-Mail: info@1a-handelsagentur.de\nInternet: www.1a-handelsagentur.de\n\nUmsatzsteueridentnummer: DE 223758473 \nWir berechnen Ihnen folgende Versandkosten:(DE) Deutschland 0,00 E...",Business Name:1a-Handelsagentur,1a HandelsagenturLindenallee 2MalchowMecklenburg17213DE,0,2,2,9,0,0,Visit the tesa Store,https://www.amazon.de/-/en/stores/tesa/page/12E699E9-FA20-4BC9-A12B-8A9CDE0C446C?ref_=ast_bln,NaN,566.0,90.0,info@1a-handelsagentur.de,932-8297,None,None,None,DE
5,15-Nov-20,Garden,Seller 6,Seller 6-link,Seller 6-storefrontlink,1-16 of 28 results,NaN,7830jeff is committed to providing each customer with the highest standard of customer service.,NaN,NaN,13,0,0,0,3284,2944,Strata,https://www.amazon.de/-/en/s/ref=bl_dp_s_web_0?ie=UTF8&search-type=ss&index=garden-de&field-keywords=Strata,28.0,NaN,NaN,None,None,None,None,None,None
14,15-Nov-20,Garden,Seller 15,Seller 15-link,Seller 15-storefrontlink,1-16 of 919 results,97% positive in the last 12 months (116 ratings),"Impressum\n \n Gesetzliche Anbieterkennung:\n \n Oliver Mills\n 9:PM\n An der Bundesstrasse 26\n 33829 Borgholzhausen\n Deutschland\n Telefon: 015140008562\n E-Mail: webmaster@9pm-store.de\n USt-IdNr.: DE215752000\n \n Wir sind seit 01.08.2015 Mitglied der Initiative ""FairCommerce"".\n Nähere Informationen hierzu finden Sie unter .fair-commerce.de. Alternative Streitbeilegung: \nDie Europäische Kommission bietet eine Onlineplattform für Streitbeilegung an, die Sie hier finden: https://ec.europa.eu/consumers/odr/\n\nUnter diesem L...",Business Name:Oliver Mills,Kaiserstr.12Borgholzhausen33829DE,15,0,0,3,1446,921,Unbekannt,https://www.amazon.de/-/en/Unbekannt/b/ref=bl_dp_s_web_19533614031?ie=UTF8&node=19533614031&field-lbr_brands_browse-bin=Unbekannt,919.0,116.0,97.0,webmaster@9pm-store.de,None,None,None,None,DE
19,15-Nov-20,Garden,Seller 20,Seller 20-link,Seller 20-storefrontlink,NaN,76% positive in the last 12 months (3721 ratings),"Impressum: ABC-Schnäppchenmarkt GmbH \n Jösser Weg 10\n 32469 Petershagen-Windheim\n Deutschland \n \n Tel.: 0570591155\n Fax: 057

In [668]:
df.size

16254

In [669]:
df.describe()

,Count of seller brands,Max % of negative seller ratings - last 30 days,Max % of negative seller ratings - last 90 days,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,product_count,rating_count,positive_rating_perc
count,602.000000,602.000000,602.000000,602.000000,602.00000,602.000000,481.000000,453.000000,559.000000
mean,9.513289,5.310631,5.735880,7.303987,4110.52990,2271.294020,6538.110187,559.735099,90.266547
std,6.736593,14.626857,12.610972,13.999249,10315.68045,6414.703705,18662.725294,2012.106260,14.258198
min,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,3.000000,10.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,5.00000,2.000000,257.000000,36.000000,89.000000
50%,14.000000,0.000000,0.000000,3.000000,456.50000,225.500000,898.000000,125.000000,94.000000
75%,15.000000,3.000000,6.000000,8.000000,2980.25000,1734.000000,3000.000000,385.000000,98.000000
max,16.000000,100.000000,100.000000,100.000000,86856.00000,70732.000000,200000.000000,37124.000000,100.000000


In [670]:
df.columns

Index(['Date Added', 'category', 'sellerlink', 'sellerlink-url',
       'sellerstorefront-url', 'sellerproductcount', 'sellerratings',
       'sellerdetails', 'seller business name', 'businessaddress',
       'Count of seller brands',
       'Max % of negative seller ratings - last 30 days',
       'Max % of negative seller ratings - last 90 days',
       'Max % of negative seller ratings - last 12 months',
       'Hero Product 1 #ratings', 'Hero Product 2 #ratings',
       'Sample brand name', 'Sample Brand URL', 'product_count',
       'rating_count', 'positive_rating_perc', 'email', 'phone_number',
       'placename', 'statename', 'zipcode', 'country'],
      dtype='object')

In [671]:
fig = go.Figure()
# fig.add_trace(go.Scatter(y=[2, 1, 4, 3]))
fig.add_trace(go.Bar(y=df['Hero Product 1 #ratings']))
fig.add_trace(go.Bar(y=df['Hero Product 2 #ratings']))
fig.update_layout(title = 'Hello Figure')
fig.show()

In [672]:
# df.sort_values('Hero Product 1 #ratings',ascending=False)

In [673]:
import plotly.express as px
fig = px.box(df, y='Hero Product 1 #ratings',x='country')
fig.add_trace(go.Box(y=df['Hero Product 2 #ratings'],x=df['country']))
fig.show()

In [674]:
df['Hero Product 1 #ratings'].groupby(df['country']).median()

country
AT      827.5
AU    10848.0
BE     2960.0
CH    25802.0
CZ    13011.0
DE      699.5
ES     2709.0
FR     1155.0
GB      930.0
HK      249.0
IE     3991.0
IN        3.0
IT      310.0
JP    45934.0
NL      404.5
PL       64.0
PT        0.0
SE     7499.0
TH      132.0
US     3368.0
Name: Hero Product 1 #ratings, dtype: float64

In [675]:
df['Hero Product 2 #ratings'].groupby(df['country']).median()

country
AT      598.0
AU     8857.0
BE     2354.0
CH     4979.0
CZ     9658.0
DE      319.5
ES     1757.0
FR      815.0
GB      339.0
HK       55.5
IE     1449.0
IN        2.0
IT      152.0
JP    20922.5
NL      243.5
PL       21.0
PT        0.0
SE     3667.0
TH      101.0
US     2229.5
Name: Hero Product 2 #ratings, dtype: float64

In [677]:
# Getting the median number of ratings for Hero Product 1 #ratings
df['Hero Product 1 #ratings'].median()

456.5

In [678]:
# Getting the median number of ratings for Hero Product 2 #ratings
df['Hero Product 2 #ratings'].median()

225.5

In [679]:
# Filter all the values with Hero Product 1 and 2 ratings above their median values
df_high_hero_ratings = df.loc[(df['Hero Product 1 #ratings'] > df['Hero Product 1 #ratings'].median()) & (df['Hero Product 2 #ratings'] > df['Hero Product 2 #ratings'].median())]

In [680]:
df_high_hero_ratings.size

7776

In [681]:
# Getting the median positive rating percentage in the last 12 months
df_high_hero_ratings.positive_rating_perc.median()

94.0

In [682]:
# Filtering all the sellers who have a % positive rating greater than median
df_high_hero_ratings = df_high_hero_ratings.loc[df_high_hero_ratings.positive_rating_perc >= df_high_hero_ratings.positive_rating_perc.median()]

In [683]:
df_high_hero_ratings.size

4077

In [689]:
# Get median product_count (if a seller has less no. of products, it's easier to manage)
df_high_hero_ratings.product_count.median()

# Let's filter all the sellers who have less than median no. of products
df_less_products = df_high_hero_ratings.loc[df_high_hero_ratings.product_count < df_high_hero_ratings.product_count.median()]

In [690]:
df_less_products.size

1890

In [693]:
df_less_products.sort_values('Hero Product 1 #ratings',ascending=False)

,Date Added,category,sellerlink,sellerlink-url,sellerstorefront-url,sellerproductcount,sellerratings,sellerdetails,seller business name,businessaddress,Count of seller brands,Max % of negative seller ratings - last 30 days,Max % of negative seller ratings - last 90 days,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,Sample brand name,Sample Brand URL,product_count,rating_count,positive_rating_perc,email,phone_number,placename,statename,zipcode,country
208,15-Nov-20,Garden,Seller 209,Seller 209-link,Seller 209-storefrontlink,1-16 of 836 results,100% positive in the last 12 months (35 ratings),IMPRESSUMBrigitte FinkErlenweg 149413 DinklageDeutschlandTel: 04443-91217E-Mail: buchfink.brigitte.fink@googlemail.comUmsatzsteuer-Identifikationsnummer gemäß § 27 a Umsatzsteuergesetz: DE 262247716Die Europäische Kommission bietet eine Onlineplattform für Streitbeilegung anUnter diesem Link finden Sie die Kontaktdaten der offiziellen Streitbeilegungsstellen: https://webgate.ec.europa.eu/odr/main/index.cfm?event=main.adr.show,Business Name:Brigitte Fink,Erlenweg1Dinklage49413DE,14,0,0,0,11917,5800,Zippo,https://www.amazon.de/-/en/Zippo/b/ref=bl_dp_s_web_1682851031?ie=UTF8&node=1682851031&field-lbr_brands_browse-bin=Zippo,836.0,35.0,100.0,buchfink.brigitte.fink@googlemail.comUmsatzsteuer-Identifikationsnummer,443-9121,None,None,None,DE
847,15-Nov-20,Garden,Seller 848,Seller 848-link,Seller 848-storefrontlink,1-16 of 827 results,98% positive in the last 12 months (251 ratings),"Die Europäische Kommission bietet eine Onlineplattform für Streitbeilegung an, die Sie hier finden: https://ec.europa.eu/consumers/odr/Unter diesem Link finden Sie die Kontaktdaten der offiziellen Streitbeilegungsstellen: https://webgate.ec.europa.eu/odr/main/index.cfm?event=main.adr.show",Business Name:VF Logistik & Handelsagentur,Wallenroder Str.7-9Berlin13435DE,13,5,2,2,10139,9398,Visit the Yankee Candle Store,https://www.amazon.de/-/en/stores/Yankee+Candle/page/B212CDCC-6B3A-488C-AC24-8C11FAE6DB7A?ref_=ast_bln,827.0,251.0,98.0,None,None,None,None,None,DE
172,15-Nov-20,Garden,Seller 173,Seller 173-link,Seller 173-storefrontlink,1-16 of 444 results,100% positive lifetime (18 total ratings),Kook Shop is committed to providing each customer with the highest standard of customer service.,Business Name:KOOK TIME PRODUCTS S.L.,AVDA CIENTIFIC AVEL.LI CORMA 25 A13MONCOFACASTELLON12593ES,14,0,0,0,9265,3008,Visit the Mepal Store,https://www.amazon.de/-/en/stores/Mepal/page/A296B2E0-8264-43D0-B41A-70F2C9596C5F?ref_=ast_bln,444.0,NaN,100.0,None,None,None,None,None,ES
154,15-Nov-20,Garden,Seller 155,Seller 155-link,Seller 155-storefrontlink,1-16 of 310 results,94% positive in the last 12 months (123 ratings),"Kobalz GmbH\n Kamenzer Str. 4\n 02997 Wittichenau\n Deutschland \n \n Tel.: 035725-79826\n Fax: 035725-79825\n E-Mail: info@kobalz.de\n Registergericht: Dresden \n Registernummer: HRB 33296 \n Geschäftsführer: Steffen Kobalz \n Umsatzsteuer-Identifikationsnummer gemäß § 27 a Umsatzsteuergesetz: DE294677856 \n \nWir sind zur Teilnahme an einem Streitbeilegungsverfahren vor einer Verbraucherschlichtungsstelle weder verpflichtet noch bereit. \n ""Klickbarer Link zur OS-Plattform (Streitbeilegung) unter ""Mehr lesen""""\n\nDie Europäis...",Business Name:Kobalz GmbH,Kobalz GmbHKamenzer Str. 4wittichenauSachsen02997DE,16,0,7,5,9265,3008,Visit the Mepal Store,https://www.amazon.de/stores/Mepal/page/A296B2E0-8264-43D0-B41A-70F2C9596C5F?ref_=ast_bln,310.0,123.0,94.0,info@kobalz.de,725-7982,None,None,None,DE
852,15-Nov-20,Garden,Seller 853,Seller 853-link,Seller 853-storefrontlink,1-16 of 364 results,98% positive in the last 12 months (996 ratings),"Impressum & Kontakt Impressum Angaben gemäß § 5 TMG Gesetzliche Anbieterkennung:\n \n Viola Direkt GmbH\n diese vertreten durch die Geschäftsführer: Vincenzo Viola, Claudio Viola\n Lindenbrunnenstr. 4\n 77855 Achern\n Deutschland\n Telefon: 07841628433\n Telefax: 07841628435\n E-Mail: sh

Things to consider
1. Less no. of products to manage
2. More positive ratings
3. High no. of ratings (high sales), or low no. of ratings (at initial stages)
4. Product count is not much useful/misleading